# Sentence Classification with Transformers
In this exercise you will implement a [Transformer](https://arxiv.org/pdf/1706.03762.pdf) and use it to judge the grammaticality of English sentences.

**A quick note: if you receive the following TypeError "super(type, obj): obj must be an instance or subtype of type", try restarting your kernel and re-running all cells.** Once you have finished making changes to the model constuctor, you can avoid this issue by commenting out all of the model instantiations after the first (e.g. lines starting with "model = ClassificationTransformer").

In [166]:
import numpy as np
import csv
import torch
from transformer import ClassificationTransformer

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython

## The Corpus of Linguistic Acceptability (CoLA)

The Corpus of Linguistic Acceptability ([CoLA](https://nyu-mll.github.io/CoLA/)) in its full form consists of 10657 sentences from 23 linguistics publications, expertly annotated for acceptability (grammaticality) by their original authors. Native English speakers consistently report a sharp contrast in acceptability between pairs of sentences. 
Some examples include:

`What did Betsy paint a picture of?` (Correct)

`What was a picture of painted by Betsy?` (Incorrect)

You can read more info about the dataset [here](https://arxiv.org/pdf/1805.12471.pdf). This is a binary classification task (predict 1 for correct grammar and 0 otherwise).

Can we train a neural network to accurately predict these human acceptability judgements? In this assignment, we will implement the forward pass of the Transformer architecture discussed in class. The general intuitive notion is that we will _encode_ the sequence of tokens in the sentence, and then predict a binary output based on the final state that is the output of the model.

## Load the preprocessed data

We've appended a "CLS" token to the beginning of each sequence, which can be used to make predictions. The benefit of appending this token to the beginning of the sequence (rather than the end) is that we can extract it quite easily (we don't need to remove paddings and figure out the length of each individual sequence in the batch). We'll come back to this.

We've additionally already constructed a vocabulary and converted all of the strings of tokens into integers which can be used for vocabulary lookup for you. Feel free to explore the data here.

In [167]:
train_inxs = np.load('../testDataset/datasets/train_inxs.npy')
val_inxs = np.load('../testDataset/datasets/val_inxs.npy')
train_labels = np.load('../testDataset/datasets/train_labels.npy')
val_labels = np.load('../testDataset/datasets/val_labels.npy')

# load dictionary
word_to_ix = {}
with open("../testDataset/datasets/word_to_ix.csv", "r") as f:
    reader = csv.reader(f)
    for line in reader:
        word_to_ix[line[0]] = line[1]
print("Vocabulary Size:", len(word_to_ix))
        
print(train_inxs.shape) # 7000 training instances, of (maximum/padded) length 43 words.
print(val_inxs.shape) # 1551 validation instances, of (maximum/padded) length 43 words.
print(train_labels.shape)
print(val_labels.shape)
print(train_inxs)

# load checkers
d1 = torch.load('../testDataset/datasets/d1.pt')
d2 = torch.load('../testDataset/datasets/d2.pt')
d3 = torch.load('../testDataset/datasets/d3.pt')
d4 = torch.load('../testDataset/datasets/d4.pt')

Vocabulary Size: 1542
(7000, 43)
(1551, 43)
(7000,)
(1551,)
[[   1    2    3 ... 1541 1541 1541]
 [  10   12    0 ... 1541 1541 1541]
 [  10   12    0 ... 1541 1541 1541]
 ...
 [ 162  409  104 ... 1541 1541 1541]
 [   8 1522   38 ... 1541 1541 1541]
 [   8 1522   38 ... 1541 1541 1541]]


## Transformers

We will be implementing a one-layer Transformer **encoder** which, similar to an RNN, can encode a sequence of inputs and produce a final output state for classification. This is the architecture:

![imgs/encoder.png](imgs/encoder.png)


You can refer to the [original paper](https://arxiv.org/pdf/1706.03762.pdf) for more details.

Instead of using numpy for this model, we will be using Pytorch to implement the forward pass. You will not need to implement the backward pass for the various layers in this assigment.

The file `gt_7643/transformer.py` contains the model class and methods for each layer. This is where you will write your implementations.

## Deliverable 1: Embeddings

We will format our input embeddings similarly to how they are constructed in [BERT (source of figure)](https://arxiv.org/pdf/1810.04805.pdf). Recall from lecture that unlike a RNN, a Transformer does not include any positional information about the order in which the words in the sentence occur. Because of this, we need to append a positional encoding token at each position. (We will ignore the segment embeddings and [SEP] token here, since we are only encoding one sentence at a time). We have already appended the [CLS] token for you in the previous step.
![imgs/embedding.png](imgs/embedding.png)

Your first task is to implement the embedding lookup, including the addition of positional encodings. Open the file `gt_7643/transformer.py` and complete all code parts for `Deliverable 1`.

In [168]:
model = ClassificationTransformer(word_to_ix, hidden_dim=128, num_heads=2, dim_feedforward=2048, dim_k=96, 
                                  dim_v=96, dim_q=96, max_length=train_inxs.shape[1])

In [169]:
inputs = train_inxs[0:2]
inputs = torch.LongTensor(inputs)

embeds = model.embed(inputs)

try:
    print("Difference:", torch.sum(torch.pairwise_distance(embeds, d1)).item()) # should be very small (<0.01)
except:
    print("NOT IMPLEMENTED")

Difference: 0.0017998494440689683


## Deliverable 2: Multi-head Self-Attention

Attention can be computed in matrix-form using the following formula:
![imgs/attn.png](imgs/attn.png)

We want to have multiple self-attention operations, computed in parallel. Each of these is called a *head*. We concatenate the heads and multiply them with the matrix `attention_head_projection` to produce the output of this layer.

After every multi-head self-attention and feedforward layer, there is a residual connection + layer normalization. Make sure to implement this, using the following formula: 
![imgs/layer_norm.png](imgs/layer_norm.png)


Open the file `gt_7643/transformer.py` and implement the `multihead_attention` function. 
We have already initialized all of the layers you will need in the constructor.

### hidden_states = model.multi_head_attention(embeds)

try:
    print("Difference:", torch.sum(torch.pairwise_distance(hidden_states, d2)).item()) # should be very small (<0.01)
except:
    print("NOT IMPLEMENTED")

In [170]:
hidden_states = model.multi_head_attention(embeds)


try: 
    print("hidden States = "+str(hidden_states))
    print("Difference:", torch.sum(torch.pairwise_distance(hidden_states, d2)).item()) # should be very small (<0.01) 

except:
    print("NOT IMPLEMENTED")

Outputs For Loop = tensor([[[-0.3771, -1.1467, -2.0995,  ...,  0.9696,  0.9097,  0.4216],
         [ 0.0756,  0.0932,  0.0486,  ...,  1.1972, -1.1659, -0.1946],
         [ 0.0150,  1.2614,  1.5895,  ..., -0.0174,  1.8849,  0.5268],
         ...,
         [ 1.6713,  0.2529,  0.5906,  ..., -0.7817,  2.2803,  0.5396],
         [ 1.9252, -0.6892,  1.2710,  ..., -0.4321,  0.6582,  1.4264],
         [-0.7143, -0.6609, -0.0183,  ..., -0.0291,  0.5226,  1.6581]],

        [[ 0.0870, -0.2262, -0.8186,  ...,  0.5323, -1.9052,  0.9277],
         [-0.3066,  1.7310, -0.5981,  ...,  1.3079, -1.5820, -1.2624],
         [-0.7623,  0.1428,  0.5490,  ...,  1.8073,  1.6597,  2.3999],
         ...,
         [ 1.6616,  0.3300,  0.5902,  ..., -0.7677,  2.2880,  0.5579],
         [ 1.9168, -0.6393,  1.3020,  ..., -0.3595,  0.6284,  1.3772],
         [-0.6979, -0.6479,  0.0150,  ...,  0.0272,  0.5380,  1.6306]]],
       grad_fn=<NativeLayerNormBackward>)
Outputs no loop = tensor([[[-0.4551, -1.7019, -2.2878, 

## Deliverable 3: Element-Wise Feed-forward Layer

Open the file `gt_7643/transformer.py` and complete code for `Deliverable 3`: the element-wise feed-forward layer consisting of two linear transformers with a ReLU layer in between.

![imgs/ffn.png](imgs/ffn.png)


In [6]:
outputs = model.feedforward_layer(hidden_states)

try:
    print("Difference:", torch.sum(torch.pairwise_distance(outputs, d3)).item()) # should be very small (<0.01)
except:
    print("NOT IMPLEMENTED")

Difference: 0.0017147620674222708


## Deliverable 4: Final Layer

Open the file `gt_7643/transformer.py` and complete code for `Deliverable 4`, to produce binary classification scores for the inputs based on the output of the Transformer.

In [7]:

scores = model.final_layer(outputs)

try:
    print("Difference:", torch.sum(torch.pairwise_distance(scores, d4)).item()) # should be very small (<1e-5)
except:
    print("NOT IMPLEMENTED")

Difference: 1.8956918665935518e-06


## Deliverable 5: Putting it all together

Open the file `gt_7643/transformer.py` and complete the method `forward`, by putting together all of the methods you have developed in the right order to perform a full forward pass.


In [8]:
inputs = train_inxs[0:2]
inputs = torch.LongTensor(inputs)

outputs = model.forward(inputs)

try:
    print("Difference:", torch.sum(torch.pairwise_distance(outputs, scores)).item()) # should be very small (<1e-5)
except:
    print("NOT IMPLEMENTED")

Difference: 1.9999999949504854e-06


Great! We've just implemented a Transformer forward pass for text classification. One of the big perks of using PyTorch is that with a simple training loop, we can rely on automatic differentation ([autograd](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)) to do the work of the backward pass for us. This is not required for this assignment, but you can explore this on your own.

Make sure when you submit your PDF for this assignment to also include a copy of `transformer.py` converted to PDF as well.